# ĐỒ ÁN XỬ LÝ NGÔN NGỮ TỰ NHIÊN
## Dịch máy Anh-Pháp với mô hình Encoder-Decoder LSTM

**HK1 / 2025-2026**

---

### Mục tiêu:
- Xây dựng mô hình **Encoder-Decoder LSTM** từ đầu
- Sử dụng **context vector cố định** (không attention)
- Dịch từ **tiếng Anh sang tiếng Pháp**
- Dataset: **Multi30K (en-fr)**
- Đánh giá: **BLEU score**

---

## 1. Cài đặt thư viện

In [ ]:
# Cài đặt các thư viện cần thiết
!pip install torch torchtext spacy nltk matplotlib seaborn tqdm -q

# Download spaCy models
!python -m spacy download en_core_web_sm -q
!python -m spacy download fr_core_news_sm -q

print("✓ Đã cài đặt xong các thư viện!")

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Import các modules đã tạo
import sys
sys.path.append('src')

from src.data_utils import get_tokenizers, Vocabulary, build_vocab_from_iterator, get_data_loaders
from src.model import Encoder, Decoder, Seq2Seq
from src.train import train, count_parameters, initialize_weights, load_checkpoint
from src.evaluate import calculate_bleu_score, plot_training_history, plot_bleu_scores, analyze_translation_errors, save_error_analysis
from src.translate import translate, greedy_decode, beam_search_decode, interactive_translation

print("✓ Import thành công!")

✓ Import thành công!


## 3. Thiết lập seed và device

In [3]:
# Set seed cho reproducibility
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Device: cpu


## 4. Tải và chuẩn bị dữ liệu
### 4.1. Download dataset Multi30K

In [4]:
from torchtext.datasets import Multi30k

print("Downloading Multi30K dataset...")

# Download dataset
# Note: Multi30K có thể cần download thủ công nếu link không khả dụng
try:
    train_data, val_data, test_data = Multi30k(split=('train', 'valid', 'test'), language_pair=('en', 'fr'))
    
    # Convert to list
    train_data = list(train_data)
    val_data = list(val_data)
    test_data = list(test_data)
    
    print(f"✓ Train: {len(train_data)} samples")
    print(f"✓ Validation: {len(val_data)} samples")
    print(f"✓ Test: {len(test_data)} samples")
    
    # Hiển thị ví dụ
    print("\nVí dụ:")
    en, fr = train_data[0]
    print(f"English: {en}")
    print(f"French:  {fr}")
    
except Exception as e:
    print(f"⚠ Lỗi khi download: {e}")
    print("\nHướng dẫn download thủ công:")
    print("1. Truy cập: https://github.com/multi30k/dataset")
    print("2. Download các file: train.en, train.fr, val.en, val.fr, test.en, test.fr")
    print("3. Đặt vào thư mục data/multi30k/")
    print("4. Uncomment đoạn code dưới để load từ file local")
    
    # # BACKUP: Load từ file local (nếu download thủ công)
    # def load_data_from_files(base_path='data/multi30k'):
    #     def read_file(path):
    #         with open(path, 'r', encoding='utf-8') as f:
    #             return [line.strip() for line in f]
    #     
    #     train_en = read_file(f'{base_path}/train.en')
    #     train_fr = read_file(f'{base_path}/train.fr')
    #     val_en = read_file(f'{base_path}/val.en')
    #     val_fr = read_file(f'{base_path}/val.fr')
    #     test_en = read_file(f'{base_path}/test.en')
    #     test_fr = read_file(f'{base_path}/test.fr')
    #     
    #     return (list(zip(train_en, train_fr)), 
    #             list(zip(val_en, val_fr)), 
    #             list(zip(test_en, test_fr)))
    # 
    # train_data, val_data, test_data = load_data_from_files()

OSError: Could not load this library: D:\ĐỒ ÁN XỬ LÍ NGÔN NGỮ TỰ NHIÊN\venv\Lib\site-packages\torchtext\lib\libtorchtext.pyd

### 4.2. Tokenization

In [ ]:
print("Khởi tạo tokenizers...")
tokenize_en, tokenize_fr = get_tokenizers()

# Test tokenizer
test_en = "A man sitting on a bench."
test_fr = "Un homme assis sur un banc."

print(f"\nEnglish: {test_en}")
print(f"Tokens:  {tokenize_en(test_en)}")
print(f"\nFrench: {test_fr}")
print(f"Tokens: {tokenize_fr(test_fr)}")

### 4.3. Xây dựng từ điển (Vocabulary)

In [ ]:
print("Xây dựng vocabulary...")

# Tách source và target
train_en = [pair[0] for pair in train_data]
train_fr = [pair[1] for pair in train_data]

# Build vocabularies (giới hạn 10,000 từ theo đề)
en_vocab = build_vocab_from_iterator(train_en, tokenize_en, max_size=10000)
fr_vocab = build_vocab_from_iterator(train_fr, tokenize_fr, max_size=10000)

print(f"\n✓ English vocabulary size: {len(en_vocab)}")
print(f"✓ French vocabulary size: {len(fr_vocab)}")

# Test numericalization
tokens = tokenize_en("hello world")
indices = en_vocab.numericalize(tokens)
print(f"\nTest: {tokens} -> {indices}")

### 4.4. Tạo DataLoaders

In [ ]:
# Hyperparameters
BATCH_SIZE = 64  # Có thể điều chỉnh: 32-128

print(f"Tạo DataLoaders với batch size = {BATCH_SIZE}...")

train_loader, val_loader, test_loader = get_data_loaders(
    train_data, val_data, test_data,
    en_vocab, fr_vocab,
    tokenize_en, tokenize_fr,
    batch_size=BATCH_SIZE,
    num_workers=0  # Tăng nếu có nhiều CPU cores
)

print(f"✓ Train batches: {len(train_loader)}")
print(f"✓ Val batches: {len(val_loader)}")
print(f"✓ Test batches: {len(test_loader)}")

# Test một batch
src, src_lengths, tgt = next(iter(train_loader))
print(f"\nBatch shapes:")
print(f"  Source: {src.shape}       (batch_size, max_src_len)")
print(f"  Lengths: {src_lengths.shape}  (batch_size,)")
print(f"  Target: {tgt.shape}       (batch_size, max_tgt_len)")

## 5. Xây dựng mô hình
### 5.1. Định nghĩa hyperparameters

In [ ]:
# Model hyperparameters (theo đề tài)
INPUT_DIM = len(en_vocab)      # Kích thước vocab tiếng Anh
OUTPUT_DIM = len(fr_vocab)     # Kích thước vocab tiếng Pháp
ENC_EMB_DIM = 256              # Embedding dimension for encoder (256-512)
DEC_EMB_DIM = 256              # Embedding dimension for decoder
HIDDEN_SIZE = 512              # Hidden size (512)
NUM_LAYERS = 2                 # Số layer LSTM (2)
DROPOUT = 0.5                  # Dropout rate (0.3-0.5)

print("Model Configuration:")
print(f"  Input vocabulary:  {INPUT_DIM}")
print(f"  Output vocabulary: {OUTPUT_DIM}")
print(f"  Embedding dim:     {ENC_EMB_DIM}")
print(f"  Hidden size:       {HIDDEN_SIZE}")
print(f"  Num layers:        {NUM_LAYERS}")
print(f"  Dropout:           {DROPOUT}")

### 5.2. Khởi tạo Encoder, Decoder, Seq2Seq

In [ ]:
# Khởi tạo Encoder
encoder = Encoder(
    input_size=INPUT_DIM,
    embedding_dim=ENC_EMB_DIM,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT
)

# Khởi tạo Decoder
decoder = Decoder(
    output_size=OUTPUT_DIM,
    embedding_dim=DEC_EMB_DIM,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT
)

# Khởi tạo Seq2Seq
model = Seq2Seq(encoder, decoder, device).to(device)

# Khởi tạo weights
initialize_weights(model)

print("✓ Model đã được khởi tạo!")
print(f"✓ Tổng số parameters: {count_parameters(model):,}")

# Hiển thị kiến trúc
print("\nModel Architecture:")
print(model)

## 6. Huấn luyện mô hình
### 6.1. Định nghĩa optimizer, loss, scheduler

In [ ]:
# Training hyperparameters
LEARNING_RATE = 0.001          # Learning rate (theo đề: Adam lr=0.001)
NUM_EPOCHS = 20                # Số epochs tối đa (10-20)
CLIP = 1.0                     # Gradient clipping
TEACHER_FORCING_RATIO = 0.5    # Teacher forcing ratio (0.5)
PATIENCE = 3                   # Early stopping patience

# Optimizer: Adam
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Loss function: CrossEntropyLoss (ignore padding)
PAD_IDX = fr_vocab.stoi[fr_vocab.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Learning rate scheduler: ReduceLROnPlateau
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2, verbose=True
)

print("Training Configuration:")
print(f"  Optimizer:             Adam (lr={LEARNING_RATE})")
print(f"  Loss:                  CrossEntropyLoss")
print(f"  Scheduler:             ReduceLROnPlateau")
print(f"  Epochs:                {NUM_EPOCHS}")
print(f"  Gradient clipping:     {CLIP}")
print(f"  Teacher forcing ratio: {TEACHER_FORCING_RATIO}")
print(f"  Early stopping:        {PATIENCE} epochs")

### 6.2. Bắt đầu training

In [ ]:
# Train model
history = train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_epochs=NUM_EPOCHS,
    clip=CLIP,
    teacher_forcing_ratio=TEACHER_FORCING_RATIO,
    device=device,
    checkpoint_dir='checkpoints',
    patience=PATIENCE
)

### 6.3. Visualize training history

In [ ]:
# Plot training history
plot_training_history(history, save_path='results/training_history.png')

## 7. Load best model

In [ ]:
# Load best checkpoint
model, checkpoint = load_checkpoint(model, 'checkpoints/best_model.pth', device)

print("\n✓ Best model đã được load!")

## 8. Đánh giá mô hình
### 8.1. Tính BLEU score trên test set

In [ ]:
from src.evaluate import calculate_bleu_score
from src.translate import translate_sentence
from nltk.translate.bleu_score import SmoothingFunction
import nltk

# Download nltk data
nltk.download('punkt', quiet=True)

print("Đang tính BLEU score trên test set...")
print("(Quá trình này có thể mất vài phút)\n")

references = []
hypotheses = []

# Giới hạn số lượng để test nhanh (hoặc dùng toàn bộ test_data)
num_samples = min(1000, len(test_data))  # Tăng lên max để có kết quả chính xác

for i, (src, tgt) in enumerate(test_data[:num_samples]):
    if i % 100 == 0:
        print(f"Progress: {i}/{num_samples}")
    
    # Dịch câu
    translated = translate_sentence(
        model, src, en_vocab, fr_vocab, tokenize_en, device, max_len=50
    )
    
    # Tokenize reference
    ref_tokens = tokenize_fr(tgt)
    
    # Tokenize hypothesis
    hyp_tokens = translated.split()
    
    references.append(ref_tokens)
    hypotheses.append(hyp_tokens)

# Tính BLEU scores
bleu_scores = calculate_bleu_score(references, hypotheses, max_n=4)

print("\n" + "="*80)
print("BLEU SCORES ON TEST SET")
print("="*80)
for metric, score in bleu_scores.items():
    print(f"{metric}: {score:.2f}%")
print("="*80)

# Lưu kết quả
import os
os.makedirs('results', exist_ok=True)
with open('results/bleu_scores.json', 'w') as f:
    json.dump(bleu_scores, f, indent=2)

print("\n✓ BLEU scores đã lưu tại: results/bleu_scores.json")

### 8.2. Visualize BLEU scores

In [ ]:
# Plot BLEU scores
plot_bleu_scores(bleu_scores, save_path='results/bleu_scores.png')

## 9. Phân tích lỗi dịch thuật

In [ ]:
print("Phân tích lỗi dịch thuật...\n")

# Phân tích 5 ví dụ lỗi
error_examples = analyze_translation_errors(
    model=model,
    test_data=test_data,
    src_vocab=en_vocab,
    tgt_vocab=fr_vocab,
    src_tokenizer=tokenize_en,
    device=device,
    num_examples=5
)

# Lưu kết quả
save_error_analysis(error_examples, save_path='results/error_analysis.json')

## 10. Test hàm translate() với câu mới

In [ ]:
# Test với một số câu mới
test_sentences = [
    "A man is sitting on a bench.",
    "The cat is sleeping on the sofa.",
    "Children are playing in the garden.",
    "She is reading a book.",
    "The weather is beautiful today."
]

print("\n" + "="*80)
print("TEST DỊCH CÂU MỚI")
print("="*80 + "\n")

for sentence in test_sentences:
    # Greedy decoding
    greedy_trans = translate(
        sentence, model, en_vocab, fr_vocab, tokenize_en, device, method='greedy'
    )
    
    # Beam search
    beam_trans = translate(
        sentence, model, en_vocab, fr_vocab, tokenize_en, device, method='beam', beam_size=3
    )
    
    print(f"English:        {sentence}")
    print(f"French (Greedy): {greedy_trans}")
    print(f"French (Beam):   {beam_trans}")
    print("-" * 80)

print("\n✓ Hàm translate() hoạt động tốt!")

## 11. Chế độ dịch tương tác (Optional)

In [ ]:
# Uncomment để chạy chế độ interactive
# interactive_translation(model, en_vocab, fr_vocab, tokenize_en, device)

## 12. Tổng kết
### 12.1. Kết quả đạt được

In [ ]:
print("\n" + "="*80)
print("TỔNG KẾT DỰ ÁN")
print("="*80)

print("\n1. MODEL ARCHITECTURE:")
print(f"   - Encoder-Decoder LSTM với context vector cố định")
print(f"   - Hidden size: {HIDDEN_SIZE}")
print(f"   - Num layers: {NUM_LAYERS}")
print(f"   - Dropout: {DROPOUT}")
print(f"   - Total parameters: {count_parameters(model):,}")

print("\n2. DATASET:")
print(f"   - Multi30K (en-fr)")
print(f"   - Train: {len(train_data)} samples")
print(f"   - Validation: {len(val_data)} samples")
print(f"   - Test: {len(test_data)} samples")

print("\n3. TRAINING:")
print(f"   - Best validation loss: {checkpoint['best_val_loss']:.4f}")
print(f"   - Training epochs: {checkpoint['epoch'] + 1}")
print(f"   - Optimizer: Adam (lr={LEARNING_RATE})")
print(f"   - Early stopping: Yes (patience={PATIENCE})")

print("\n4. BLEU SCORES:")
for metric, score in bleu_scores.items():
    print(f"   - {metric}: {score:.2f}%")

print("\n5. FILES GENERATED:")
print(f"   ✓ checkpoints/best_model.pth")
print(f"   ✓ results/training_history.json")
print(f"   ✓ results/training_history.png")
print(f"   ✓ results/bleu_scores.json")
print(f"   ✓ results/bleu_scores.png")
print(f"   ✓ results/error_analysis.json")

print("\n" + "="*80)
print("✓ ĐỒ ÁN HOÀN THÀNH!")
print("="*80)

print("\nTiếp theo:")
print("  1. Viết báo cáo PDF (bao gồm sơ đồ kiến trúc, biểu đồ, phân tích lỗi)")
print("  2. Đề xuất cải tiến (attention, beam search, augmentation, ...)")
print("  3. Nộp: main.ipynb + best_model.pth + report.pdf")

## 13. Đề xuất cải tiến (Bonus: +1.0 điểm)

### Các cải tiến có thể thực hiện:

1. **Thay greedy decoding bằng beam search** (beam size = 3-5)  
   ✅ Đã implement trong `src/translate.py`

2. **Thêm Attention Mechanism** (Bahdanau/Luong attention)  
   - Context vector động thay vì cố định
   - Cải thiện BLEU score đáng kể

3. **Sử dụng dataset WMT 2014** (~36 triệu câu thay vì 29k)  
   - Tăng kích thước vocab
   - Model học được nhiều pattern hơn

4. **Tăng số layer LSTM** hoặc **hidden size**  
   - 3-4 layers, hidden size 1024
   - Cần GPU mạnh hơn

5. **Subword tokenization** (BPE - Byte Pair Encoding)  
   ✅ Đề tài yêu cầu có thể thử
   - Giảm OOV (out-of-vocabulary)
   - Sử dụng thư viện `sentencepiece` hoặc `subword-nmt`

6. **So sánh hiệu suất với mô hình Transformer**  
   - Baseline: LSTM seq2seq
   - Advanced: Transformer (self-attention)

7. **Overfitting handling**  
   - Dropout, layer normalization
   - Data augmentation (back-translation)

8. **Scheduled sampling** thay vì teacher forcing cố định  
   - Giảm dần teacher forcing ratio theo epoch

---

**Lựa chọn khuyến nghị cho Bonus:**
- Thêm **Attention** (dễ implement, cải thiện rõ rệt)
- Thử **Beam search** với beam size khác nhau (3, 5, 10)
- Phân tích sâu hơn các lỗi dịch (ngữ pháp, ngữ nghĩa)
- So sánh với **WMT 2014** dataset

---
**HẾT**